In [1]:
import os
import pandas as pd
import numpy as np
from fractions import Fraction
import math

# Utilities

In [2]:
def WeightSeq(floor,p,q):
    initial=int(floor)+Fraction(p,q)
    start=[]
    divisor=1
    remainder=initial
    while remainder>0:
        ordered=sorted([remainder,divisor])
        divisor,remainder=ordered
        start.append(divisor)
        remainder=remainder-divisor
        #print(f"Divisor is {divisor} and remainder is {remainder}")
    return np.array(start)

def block_indices(floor,p,q):
    weight_sequence = WeightSeq(floor,p,q)
    unique_entries = set(weight_sequence)
    return [[i for i, x in enumerate(weight_sequence) if x==entry] for entry in unique_entries]

def block_multiplicity_check(block_multi_indices, tail): #block_multi_indices is output of block_indices, tail is a Series objects whose index is either the DataFrame’s index (axis=0)
    blocks = [[tail[i] for i in block_multi_index] for block_multi_index in block_multi_indices] 
    
    tail_length = max([entry for block in block_multi_indices for entry in block])
    
    block_entries = [entry for block in blocks for entry in block]
    
    if 0 in block_entries or np.isnan(sum(block_entries)): #If block is too short
        #print("Too short")
        return False
    
    number_distinct = [len(set(block)) for block in blocks]
    if sum(number_distinct) > len(blocks)+1: #Only one block can have at most two distinct entries
        #print("Too many non-uniform")
        return False
    
    max_size = max([entry for block in block_multi_indices for entry in block])
    if tail[str(tail_length+1)]!=0: #Tail can't be too long
        #print("entry",tail[str(tail_length+1)],"should be zero or empty")
        #print("Too long")
        return False

    for block in blocks:
        if sum(block)/max(block)!=len(block): #If block is not uniform
            if (len(block)*min(block)+1-sum(block))*(len(block)*max(block)-1-sum(block))!=0: #If off entry is more than +-1 different from rest
                return False
    return True

def test_obstruction_value(a,p,q,candidate_tail):
    d = candidate_tail['d']
    e = candidate_tail['e']
    weight_seq = WeightSeq(a,p,q)
    a_value = a+p/q
    print("tail is",candidate_tail['tail'])
    mu_value = np.inner(candidate_tail['tail'], weight_seq)/(d+2*e)
    v_bound = np.sqrt(a_value/2)
    if mu_value>v_bound:
        print("FOUNDVolume is",v_bound,"Obs is",mu_value)
        return True
    else:
        print("Volume is",v_bound,"Obs is",mu_value)
        return False

In [3]:
block_candidates = pd.read_parquet("compiled_solns_8.parquet")

# Generate dictionary of a-values, block patterns

In [4]:
#Limits are q=2 up until q=11, need reduced fractions, so just numerators that are coprime to originals
avalues = {}
apq = {}
for i in range(2,12):
    coprimes = [elt for elt in range(i) if math.gcd(i,elt) == 1]
    avalues[i] = coprimes
    for coprime in coprimes:
        label = str(coprime)+"_"+str(i)
        #print(block_indices(8,coprime,i))
        apq[label]={'blocks':block_indices(8,coprime,i),'a':8+coprime/i}

# Check for obstructive classes

In [60]:
for value in apq.items():
    print("Checking",value[1],"and",value[0])
    block_consistent_list= block_candidates.drop(['d','e','ell2','ell1'],axis=1).apply(lambda x: block_multiplicity_check(value[1]['blocks'],x), axis=1)
    candidates_pq = block_candidates[block_consistent_list].reset_index()
    #print(candidates_pq)
    tail_length = max([entry for block in value[1]['blocks'] for entry in block])+1
    tails_pq=candidates_pq.transpose().tail(19).head(tail_length).transpose().values
    candidates_pq['tail']=tails_pq.tolist()
    candidates_dict=candidates_pq[["d","e","tail"]].to_dict(orient="records")
    for candidate in candidates_dict:
        test_obstruction_value(8,int(value[0][0]),int(value[0][2:]),candidate)

Checking {'blocks': [[8, 9], [0, 1, 2, 3, 4, 5, 6, 7]], 'a': 8.5} and 1_2
tail is [7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 6.0, 1.0, 1.0]
Volume is 2.0615528128088303 Obs is 1.435897435897436
tail is [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 3.0, 2.0]
Volume is 2.0615528128088303 Obs is 1.4473684210526316
tail is [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 3.0, 2.0]
Volume is 2.0615528128088303 Obs is 1.4473684210526316
tail is [13.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 1.0, 1.0]
Volume is 2.0615528128088303 Obs is 1.4202898550724639
tail is [16.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 3.0, 3.0]
Volume is 2.0615528128088303 Obs is 1.441860465116279
tail is [16.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 3.0, 3.0]
Volume is 2.0615528128088303 Obs is 1.441860465116279
tail is [19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 2.0]
Volume is 2.0615528128088303 Obs is 1.4305555555555556
tail is [19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 2.0]
Volume is 2.0615